# 7-3. 영화진흥원 박스오피스 데이터 수집, 저장
# #01. 필요한 패키지 가져오기

In [1]:
import requests
import json
import datetime as dt
import pandas as pd
from pandas import DataFrame
import pymysql
from sqlalchemy import create_engine
from matplotlib import pyplot

# #02. OpenAPI를 통한 데이터 수집
## 1) API Key 준비하기

In [2]:
api_key = '498bf1773e4b92ddef9471c8f988acb3'

## 2) 세션 생성

In [3]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"
session= requests.Session()
session.headers.update({'User-agent':user_agent, 'referer':None})

## 3) 접속 주소 준비하기
### 변수가 치환될 주소 템플릿

In [4]:
url_tpl = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key={key}&targetDt={date}'

### 조회날짜 문자열 만들기
여기서는 어제 날짜로 준비

In [5]:
today = dt.datetime.now()
delta = dt.timedelta(days=-1)                  # 하루 전을 의미하는 timedelta 객체
yesterday = today+delta                        # 오늘 날짜와 timedelta 연산
yesterday_str = yesterday.strftime('%Y%m%d')   # yyyymmdd 형식 문자열로 변환
yesterday_str

'20210302'

### 최종 접속 주소 확인

In [6]:
api_url = url_tpl.format(key=api_key, date=yesterday_str)
api_url

'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=498bf1773e4b92ddef9471c8f988acb3&targetDt=20210302'

## 4) API를 통한 JSON 데이터 가져오기

In [7]:
# 데이터 가져오기
r = session.get(api_url)

# 결과 검사
if r.status_code != 200:
    print("[%d Error] %s" % (r.status_code, r.reason))
    quit()
    
r.encoding = 'utf-8'
r.text

'{"boxOfficeResult":{"boxofficeType":"일별 박스오피스","showRange":"20210302~20210302","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"2","rankOldAndNew":"OLD","movieCd":"20200703","movieNm":"극장판 귀멸의 칼날: 무한열차편","openDt":"2021-01-27","salesAmt":"134303840","salesShare":"31.9","salesInten":"-187741190","salesChange":"-58.3","salesAcc":"9020206220","audiCnt":"13931","audiInten":"-18687","audiChange":"-57.3","audiAcc":"939663","scrnCnt":"592","showCnt":"1206"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20199541","movieNm":"미션 파서블","openDt":"2021-02-17","salesAmt":"73313050","salesShare":"17.4","salesInten":"-245739580","salesChange":"-77","salesAcc":"3382593160","audiCnt":"8102","audiInten":"-25997","audiChange":"-76.2","audiAcc":"373333","scrnCnt":"625","showCnt":"1496"},{"rnum":"3","rank":"3","rankInten":"-2","rankOldAndNew":"OLD","movieCd":"20207443","movieNm":"소울","openDt":"2021-01-20","salesAmt":"63028360","salesShare":"15.0","salesInten":"-266296010","

### JSON을 딕셔너리로 변환

In [8]:
daily_boxoffice_dict = json.loads(r.text)
daily_boxoffice_dict

{'boxOfficeResult': {'boxofficeType': '일별 박스오피스',
  'showRange': '20210302~20210302',
  'dailyBoxOfficeList': [{'rnum': '1',
    'rank': '1',
    'rankInten': '2',
    'rankOldAndNew': 'OLD',
    'movieCd': '20200703',
    'movieNm': '극장판 귀멸의 칼날: 무한열차편',
    'openDt': '2021-01-27',
    'salesAmt': '134303840',
    'salesShare': '31.9',
    'salesInten': '-187741190',
    'salesChange': '-58.3',
    'salesAcc': '9020206220',
    'audiCnt': '13931',
    'audiInten': '-18687',
    'audiChange': '-57.3',
    'audiAcc': '939663',
    'scrnCnt': '592',
    'showCnt': '1206'},
   {'rnum': '2',
    'rank': '2',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20199541',
    'movieNm': '미션 파서블',
    'openDt': '2021-02-17',
    'salesAmt': '73313050',
    'salesShare': '17.4',
    'salesInten': '-245739580',
    'salesChange': '-77',
    'salesAcc': '3382593160',
    'audiCnt': '8102',
    'audiInten': '-25997',
    'audiChange': '-76.2',
    'audiAcc': '373333',
    'scrnCnt': 

### 딕셔너리를 데이터프레임으로 변환

In [9]:
daily_boxoffice_df = DataFrame(daily_boxoffice_dict['boxOfficeResult']['dailyBoxOfficeList'])
daily_boxoffice_df

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,2,OLD,20200703,극장판 귀멸의 칼날: 무한열차편,2021-01-27,134303840,31.9,-187741190,-58.3,9020206220,13931,-18687,-57.3,939663,592,1206
1,2,2,0,OLD,20199541,미션 파서블,2021-02-17,73313050,17.4,-245739580,-77,3382593160,8102,-25997,-76.2,373333,625,1496
2,3,3,-2,OLD,20207443,소울,2021-01-20,63028360,15.0,-266296010,-80.9,17803392410,6818,-27568,-80.2,1921847,591,1245
3,4,4,1,OLD,20215843,카오스 워킹,2021-02-24,39628010,9.4,-136984890,-77.6,1040542510,5130,-14930,-74.4,125016,589,1338
4,5,5,-1,OLD,20202555,톰과 제리,2021-02-24,32466920,7.7,-163410190,-83.4,995994240,3709,-18278,-83.1,117037,613,1137
5,6,6,0,OLD,20198663,고백,2021-02-24,11295600,2.7,-12168700,-51.9,181814900,1578,-996,-38.7,25127,192,350
6,7,7,2,OLD,20215221,더 레이서,2021-02-24,5466370,1.3,-3641970,-40,67822210,678,-445,-39.6,8271,124,145
7,8,8,4,OLD,20215862,라스트 레터,2021-02-24,4561900,1.1,-3914500,-46.2,69326710,567,-379,-40.1,8165,109,181
8,9,9,-1,OLD,20214848,퍼펙트 케어,2021-02-19,4493010,1.1,-9068070,-66.9,199303090,545,-913,-62.6,23579,85,97
9,10,10,-3,OLD,20209013,워 위드 그랜파,2021-02-24,4438880,1.1,-14073180,-76,106842170,536,-1532,-74.1,12983,194,246


# #05. 데이터 저장하기
## 1) 데이터베이스 접속하기
### 데이터베이스 연동에 필요한 접속 정보

In [10]:
HOSTNAME = 'localhost'
PORT = 3306
USERNAME = 'root'
PASSWORD = '123qwe!@#'
DATABASE = 'myschool'
CHARSET1 = 'utf8'          # MySQL에서 사용할 캐릭터셋
CHARSET2 = 'utf-8'         # Python에서 사용할 캐릭터셋

### 접속 문자열 생성
mysql+mysqldb://아이디:비밀번호@DB서버주소:포트번호/DB이름?charset=인코딩

In [11]:
con_str_fmt = "mysql+mysqldb://{0}:{1}@{2}:{3}/{4}?charset={5}"
con_str = con_str_fmt.format(USERNAME, PASSWORD, HOSTNAME, PORT, DATABASE, CHARSET1)
con_str

'mysql+mysqldb://root:123qwe!@#@localhost:3306/myschool?charset=utf8'

### pymysql을 사용하여 MySQL 연동 객체 설치하기

In [12]:
pymysql.install_as_MySQLdb()
import MySQLdb

### 데이터베이스 접속

In [13]:
engine = create_engine(con_str, encoding=CHARSET2)
conn = engine.connect()

## 2) 수집한 데이터 저장하기

In [15]:
daily_boxoffice_df.to_sql(name='daily_boxoffice',
                          con=conn,
                          if_exists='append', index=False)

## 3) 데이터베이스 접속 해제

In [16]:
conn.close()